In [1]:
import sys
import numpy as np
import pandas as pd 
from skmultiflow.data import DataStream, MultilabelGenerator, ConceptDriftStream
from skmultiflow.meta import ClassifierChain
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier
from skmultiflow.bayes import NaiveBayes
from skmultiflow.metrics import hamming_score
from sklearn.metrics import accuracy_score, f1_score, hamming_loss
from ld3 import LD3
from skmultilearn.dataset import load_from_arff
from sklearn.preprocessing import MultiLabelBinarizer
from skmultiflow.drift_detection import ADWIN, EDDM, KSWIN, HDDM_W, HDDM_A, DDM, PageHinkley
from sklearn.naive_bayes import GaussianNB
import util
import warnings
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
from river.metrics import ExampleF1
from scipy.stats import truncnorm
from tornados.drift_detection import fhddm, fhddms, fhddms_add, mddm_a, mddm_e, mddm_g, rddm, seq_drift2
from ld3_ import LD3, Window, StreamGenerator

warnings.filterwarnings('ignore')
np.set_printoptions(precision=4, suppress=True)

In [7]:
'''n_features = 100
n_targets = 20
s1 = MultilabelGenerator(n_samples=6000, n_features=n_features, n_targets=n_targets, n_labels=1, random_state=100) #100
s2 = MultilabelGenerator(n_samples=6009, n_features=n_features, n_targets=n_targets, n_labels=2, random_state=250)
stream = ConceptDriftStream(stream=s1, drift_stream=s2, position=4000, width=1, random_state=0)
sample_size=10000'''

#gen = StreamGenerator()
#stream, sample_size, n_features, n_targets = gen.get_stream(type='gradual1')

X, y = load_from_arff('./datasets/{}'.format('Mediamill.arff'), label_count=101)
X = X.toarray()
y = y.toarray().astype(np.int8)
#X, y = util.induce_drift(X, y, 10000, 19300, 20, 1006, percentage=2)
sample_size = len(X)
n_features = X.shape[1]

if len(np.unique(y)) > 2:
    mlb = MultiLabelBinarizer()
    y = mlb.fit_transform(y)

n_targets = y.shape[1]
stream = DataStream(data = X, y=y, n_targets=n_targets)

In [12]:
thresh = [0.9,1,1.1,1.2]
for t in thresh:
    detector = LD3(window_size=250, correlation_thresh=t, bandwith=0.1, float_decimal=4) #20NG-0.06   Synthetic_sudden-0.01    tmc 0.4-4/5     imbd 0.15     ohsumed 0.5
    clf = ClassifierChain(GaussianNB())#PassiveAggressiveClassifier(random_state=0))
    pre_sample = [np.zeros(n_features), np.zeros(n_features)]
    pre_label = [np.ones(n_targets), np.zeros(n_targets)]
    clf.fit(np.array(pre_sample), np.array(pre_label))

    stream.restart()
    max_samples = sample_size
    y_true = []
    y_pred = []
    pretrain_X = []
    pretrain_y = []
    p_bar = tqdm(total=max_samples)
    n_samples = 0
    drift, warning = False, False
    correlation = 0
    w_x = Window(max_size=199)
    w_y = Window(max_size=199)
    while n_samples < max_samples and stream.has_more_samples():
        X, y = stream.next_sample()
        w_x.queue(X.flatten())
        #w_y.queue(y)
        w_y.queue(y.flatten())
        '''if n_samples % 60 == 0 and n_samples > 0:
            clf.reset()
            clf.fit(np.array(pre_sample), np.array(pre_label))
            try:
                clf.fit(w_x.get_window, w_y.get_window)
            except:
                clf.fit(np.array(pre_sample), np.array(pre_label))'''
                
        '''if n_samples >= 4000 and n_samples <= 4500:
            print('Correlation @', n_samples, ': ', score, correlation)
            #print(detector._average_correlation)
            #r1, r2 = detector._ranks
            #print('[',','.join(list(map(str,r1))), ']', '\t[', ','.join(list(map(str,r2))), ']')'''
        if drift:
            print('Drift@', n_samples)
            r1, r2 = detector._ranks
            #print('[',','.join(list(map(str,r1))), ']', '\t[', ','.join(list(map(str,r2))), ']')
            clf.reset()
            clf.fit(np.array(pre_sample), np.array(pre_label))
            '''try:
                clf.fit(w_x.get_window, w_y.get_window)
            except:
                w_x.queue(pre_sample[0])
                w_x.queue(pre_sample[1])
                w_y.queue(pre_label[0])
                w_y.queue(pre_label[1])
                clf.fit(w_x.get_window, w_y.get_window)'''

        pred = clf.predict(X)
        #clf.partial_fit(X, np.array([y]))
        clf.partial_fit(X, y)
        drift, warning, correlation, score = detector.update(pred.astype(np.int32))
        #detector.add_element(np.all(pred.flatten() == y))

        #y_true.append(y)
        y_true.extend(y)
        y_pred.extend(pred)

        p_bar.update(1)
        n_samples += 1
    p_bar.close()
    stream.restart()


    print()
    print(np.round(util.accuracy_example(np.array([y_true]), np.array([y_pred])), decimals=4)) # 
    #print(np.round(accuracy_score(np.array(y_true), np.array(y_pred)), decimals=4))
    print(np.round(util.hamming_loss(np.array(y_true), np.array(y_pred)), decimals=4))
    print(np.round(f1_score(np.array(y_true), np.array(y_pred), average='samples'),decimals=4))#util.f1_example(np.array([y_true]), np.array([y_pred])))
    print(np.round(util.f1_micro(np.array(y_true), np.array(y_pred)), decimals=4))
    print(np.round(util.f1_macro(np.array(y_true), np.array(y_pred)), decimals=4))
    print()

  0%|          | 0/43907 [00:00<?, ?it/s]


0.0545
0.5211
0.1614
0.1034
0.0672



  0%|          | 0/43907 [00:00<?, ?it/s]


0.0545
0.5211
0.1614
0.1034
0.0672



  0%|          | 0/43907 [00:00<?, ?it/s]

...


0.0545
0.5211
0.1614
0.1034
0.0672



  0%|          | 0/43907 [00:00<?, ?it/s]


0.0545
0.5211
0.1614
0.1034
0.0672



  0%|          | 0/43907 [00:00<?, ?it/s]


0.0545
0.5211
0.1614
0.1034
0.0672



In [9]:
detectors = [ADWIN(), EDDM(), DDM(), KSWIN(), HDDM_A(), HDDM_W()]
for detector in detectors:
    #detector = EDDM()#LD3(k=1, window_size=500)
    clf = ClassifierChain(GaussianNB())#SGDClassifier(max_iter=1000, random_state=0))#
    pre_sample = [np.zeros(n_features), np.zeros(n_features)]
    pre_label = [np.ones(n_targets), np.zeros(n_targets)]
    clf.fit(np.array(pre_sample), np.array(pre_label))

    stream.restart()
    max_samples = sample_size
    y_true = []
    y_pred = []
    pretrain_X = []
    pretrain_y = []
    p_bar = tqdm(total=max_samples)
    n_samples = 0
    w_x = Window(max_size=199)
    w_y = Window(max_size=199)
    while n_samples < max_samples and stream.has_more_samples():
        X, y = stream.next_sample()
        w_x.queue(X.flatten())
        w_y.queue(y)
        if detector.detected_change():
            print('Drift@', n_samples)
            detector.reset()
            clf.reset()
            clf.fit(np.array(pre_sample), np.array(pre_label))
            '''try:
                clf.fit(w_x.get_window, w_y.get_window)
            except:
                clf.fit(np.array(pre_sample), np.array(pre_label))'''

        pred = clf.predict(X)
        #clf.partial_fit(X, np.array([y]))
        clf.partial_fit(X, y)
        #detector.add_element(pred.astype(np.int32))
        #detector.add_element(np.all(pred.flatten() == y))
        detector.add_element((pred.astype(np.int32).flatten().tolist())==(y.astype(np.int32).flatten().tolist()))

        #y_true.append(y)
        y_true.extend(y)
        y_pred.extend(pred)

        p_bar.update(1)
        n_samples += 1
    
    print()
    print(np.round(util.accuracy_example(np.array([y_true]), np.array([y_pred])), decimals=4))
    #print(np.round(accuracy_score(np.array(y_true), np.array(y_pred)), decimals=4))
    print(np.round(util.hamming_loss(np.array(y_true), np.array(y_pred)), decimals=4))
    print(np.round(f1_score(np.array(y_true), np.array(y_pred), average='samples'),decimals=4))#util.f1_example(np.array([y_true]), np.array([y_pred])))
    print(np.round(util.f1_micro(np.array(y_true), np.array(y_pred)), decimals=4))
    print(np.round(util.f1_macro(np.array(y_true), np.array(y_pred)), decimals=4))
    print()

    stream.restart()


  0%|          | 0/43907 [00:00<?, ?it/s]

Drift@ 16383
Drift@ 23583
Drift@ 31263
Drift@ 35775

0.0765
0.2219
0.2063
0.1422
0.0611



  0%|          | 0/43907 [00:00<?, ?it/s]

Drift@ 2759
Drift@ 3806
Drift@ 5289
Drift@ 7234
Drift@ 9280
Drift@ 10719
Drift@ 11619
Drift@ 12672
Drift@ 13309
Drift@ 14575
Drift@ 15223
Drift@ 17279
Drift@ 17958
Drift@ 19757
Drift@ 21676
Drift@ 22817
Drift@ 23843
Drift@ 26729
Drift@ 28593
Drift@ 30246
Drift@ 31398
Drift@ 33225
Drift@ 34475
Drift@ 35017
Drift@ 36329
Drift@ 37990
Drift@ 39079
Drift@ 40653
Drift@ 42360

0.0591
0.2336
0.1231
0.1116
0.0548



  0%|          | 0/43907 [00:00<?, ?it/s]

...


0.0545
0.5211
0.1614
0.1034
0.0672



  0%|          | 0/43907 [00:00<?, ?it/s]


0.0545
0.5211
0.1614
0.1034
0.0672



  0%|          | 0/43907 [00:00<?, ?it/s]


0.0545
0.5211
0.1614
0.1034
0.0672



In [10]:
detectors = [fhddm.FHDDM(), fhddms.FHDDMS(), fhddms_add.FHDDMS_add(), mddm_a.MDDM_A(), mddm_e.MDDM_E(), mddm_g.MDDM_G(), seq_drift2.SeqDrift2ChangeDetector(), rddm.RDDM()]
for detector in detectors:
    #detector = EDDM()#LD3(k=1, window_size=500)
    clf = ClassifierChain(GaussianNB())#SGDClassifier(max_iter=1000, random_state=0))#
    pre_sample = [np.zeros(n_features), np.zeros(n_features)]
    pre_label = [np.ones(n_targets), np.zeros(n_targets)]
    clf.fit(np.array(pre_sample), np.array(pre_label))


    max_samples = sample_size
    y_true = []
    y_pred = []
    pretrain_X = []
    pretrain_y = []
    p_bar = tqdm(total=max_samples)
    n_samples = 0
    drift = False
    w_x = Window(max_size=199)
    w_y = Window(max_size=199)
    while n_samples < max_samples and stream.has_more_samples():
        X, y = stream.next_sample()
        w_x.queue(X.flatten())
        w_y.queue(y)
        if drift:
            print('Drift@', n_samples)
            detector.reset()
            clf.reset()
            clf.fit(np.array(pre_sample), np.array(pre_label))
            '''try:
                clf.fit(w_x.get_window, w_y.get_window)
            except:
                clf.fit(np.array(pre_sample), np.array(pre_label))'''

        pred = clf.predict(X)
        #clf.partial_fit(X, np.array([y]))
        clf.partial_fit(X, y)
        #detector.add_element(pred.astype(np.int32))
        #detector.add_element(np.all(pred.flatten() == y))
        _, drift = detector.run((pred.astype(np.int32).flatten().tolist())==(y.astype(np.int32).flatten().tolist()))

        #y_true.append(y)
        y_true.extend(y)
        y_pred.extend(pred)

        p_bar.update(1)
        n_samples += 1
    
    print()
    print(np.round(util.accuracy_example(np.array([y_true]), np.array([y_pred])), decimals=4))
    #print(np.round(accuracy_score(np.array(y_true), np.array(y_pred)), decimals=4))
    print(np.round(util.hamming_loss(np.array(y_true), np.array(y_pred)), decimals=4))
    print(np.round(f1_score(np.array(y_true), np.array(y_pred), average='samples'),decimals=4))#util.f1_example(np.array([y_true]), np.array([y_pred])))
    print(np.round(util.f1_micro(np.array(y_true), np.array(y_pred)), decimals=4))
    print(np.round(util.f1_macro(np.array(y_true), np.array(y_pred)), decimals=4))
    print()

    stream.restart()

  0%|          | 0/43907 [00:00<?, ?it/s]


0.0545
0.5211
0.1614
0.1034
0.0672



  0%|          | 0/43907 [00:00<?, ?it/s]


0.0545
0.5211
0.1614
0.1034
0.0672



  0%|          | 0/43907 [00:00<?, ?it/s]

...


0.0545
0.5211
0.1614
0.1034
0.0672



  0%|          | 0/43907 [00:00<?, ?it/s]

Drift@ 3000
Drift@ 3600

0.0555
0.4921
0.1579
0.1051
0.0668



  0%|          | 0/43907 [00:00<?, ?it/s]

Drift@ 960
Drift@ 8083
Drift@ 14766
Drift@ 19100
Drift@ 23560
Drift@ 26224
Drift@ 29381
Drift@ 38287

0.0782
0.1884
0.1886
0.145
0.0585



In [6]:
from sklearn.neighbors import KernelDensity
model = KernelDensity(bandwidth=1e-1, kernel='gaussian')
sample = np.array(correlation_list).reshape((len(correlation_list), 1))
model.fit(sample)
# sample probabilities for a range of outcomes
values = np.asarray([value for value in np.linspace(-1,1,100)])
values = values.reshape((len(values), 1))
probabilities = model.score_samples(values)
probabilities = np.exp(probabilities)
# plot the histogram and pdf
plt.hist(sample, bins=36, density=True)
plt.plot(values[:], probabilities)
plt.show()

NameError: name 'correlation_list' is not defined

In [8]:
print(accuracy_score(np.array(y_true), np.array(y_pred)))
print(hamming_score(np.array(y_true), np.array(y_pred)))
print(f1_score(np.array(y_true), np.array(y_pred), average='samples'))
print(f1_score(np.array(y_true), np.array(y_pred), average='micro'))

0.2979274611398964
0.9229740932642487
0.40461090550209716
0.38694379149655656


In [9]:
a = np.array([1, 19, 3, 8, 5, 18, 4, 2, 0, 13, 11, 7, 17, 6, 9, 10, 12, 14, 15, 16])
b = np.array([19, 1, 2, 7, 3, 5, 18, 17, 13, 4, 8, 11, 0, 10, 6, 12, 9, 14, 15, 16]) 

c = np.array([19, 1, 2, 17, 10, 18, 7, 8, 3, 12, 4, 6, 5, 13, 11, 9, 14, 15, 0, 16])
d = np.array([19, 2, 1, 8, 18, 5, 17, 3, 13, 12, 4, 6, 7, 10, 11, 9, 14, 15, 16, 0]) 

x = np.array([ 1,8,19,5,13,0,17,2,18,11,3,6,9,10,12,4,7,14,15,16 ])
y = np.array([ 19,2,1,8,18,10,12,5,3,4,7,6,11,9,13,17,14,0,15,16 ])

e = np.array([ 8,19,17,1,3,5,0,9,18,2,4,6,11,7,13,12,10,14,15,16 ])
f = np.array([ 19,1,2,8,7,18,5,3,4,13,11,10,17,0,6,12,9,14,15,16 ])

In [10]:
argx = (np.argsort(a)).astype(np.float)#np.arange(1,21,1).astype(np.float)
argy = (np.argsort(b)).astype(np.float)#np.flip(np.arange(1,21,1)).astype(np.float) 
maxx = np.arange(1,21,1).astype(np.float)
maxy = np.flip(np.arange(1,21,1)).astype(np.float) 

'''for i in range(20):
    argx[i] *= 1/(argx[i]+1)
    argy[i] *= 1/(argy[i]+1)
    maxx[i] *= 1/(maxx[i]+1)
    maxy[i] *= 1/(maxy[i]+1)'''



'for i in range(20):\n    argx[i] *= 1/(argx[i]+1)\n    argy[i] *= 1/(argy[i]+1)\n    maxx[i] *= 1/(maxx[i]+1)\n    maxy[i] *= 1/(maxy[i]+1)'

In [11]:
print(argx)
print(argy)

[ 8.  0.  7.  2.  6.  4. 13. 11.  3. 14. 15. 10. 16.  9. 17. 18. 19. 12.
  5.  1.]
[12.  1.  2.  4.  9.  5. 14.  3. 10. 16. 13. 11. 15.  8. 17. 18. 19.  7.
  6.  0.]


In [12]:
def WS(rx, ry):
    sum_ = 0
    ranks_x = np.argsort(rx)
    ranks_y = np.argsort(ry)
    for i in range(len(rx)):
        sum_ += (1/(2**(ranks_x[i]))) * ((np.abs(ranks_x[i] - ranks_y[i]))/(np.max([np.abs(1-ranks_x[i]), np.abs(len(rx) - ranks_x[i])])))
    return 1 - sum_

In [13]:
WS(e,f)

0.6403404417406884

In [14]:
a = [1,2,3,4,5]
b = [5,4,3,2,1]
WS(a, b)

-0.2583333333333333

In [15]:
from scipy.stats import kendalltau, spearmanr
spearmanr(x[:5],y[:5])

SpearmanrResult(correlation=-0.7, pvalue=0.1881204043741873)

In [16]:
from scipy.spatial.distance import correlation
correlation(argx,argy)

0.15639097744360897

In [17]:
def dist(x, y, w):
    sum_ = 0
    for i in range(len(x)):
        abs_ = np.abs(x[i] - y[i])
        add =  abs_*w[i] if abs_ > 1 else 0
        sum_ += add
    return sum_
        

In [18]:
from scipy.spatial import distance
((dist(argx, argy, w=1/((3)**(argy*argy))) / np.sqrt(len(argx))) + (dist(argx, argy, w=1/((3)**(argx*argx))) / np.sqrt(len(argx)))) / 2 #/ 0.05617535148406078

0.00974723033648846

In [19]:
def rank_correlation(x, y, dist=distance.correlation):
        c = np.array([dist(x[i], y[i]) * (1/((y[i]))) for i in range(len(x))]).sum()
        return c
rank_correlation(argx, argy)

nan

In [20]:
1/argy

array([0.0833, 1.    , 0.5   , 0.25  , 0.1111, 0.2   , 0.0714, 0.3333,
       0.1   , 0.0625, 0.0769, 0.0909, 0.0667, 0.125 , 0.0588, 0.0556,
       0.0526, 0.1429, 0.1667,    inf])

In [21]:
from scipy.spatial import distance
dist = distance.cityblock # * 1/((argy[i])**2)
s = np.array([dist(argx[i], argy[i]) * 1/((argy[i])**2) for i in range(20)]).sum()

'''for i in range(20):
    s += dist(argx[i], argy[i]) * 1/((argy[i])**2)'''

smax = 0
for i in range(20):
    smax += dist(argx[i], argx[i]) * 1/((argy[i])**2)

print(s)

inf


In [22]:
0.5429113076754074
55.77
weigher = lambda r: 1/((1+r)**2)
print(weightedtau(a,b,rank=True, additive=False, weigher=weigher))
print(weightedtau(c,d,rank=True, additive=False, weigher=weigher))
print(weightedtau(x,y,rank=True, additive=False, weigher=weigher))

NameError: name 'weightedtau' is not defined

In [19]:
print(np.where(a==b)[0])
print(np.where(c==d)[0])
print(np.where(x==y)[0])

[17 18 19]
[ 1 10]
[11 18 19]


In [20]:
weigher = lambda r: 1/((r+1))
print(weightedtau(np.argsort(a),np.argsort(b),rank=False, additive=False, weigher=weigher))
print(weightedtau(np.argsort(c),np.argsort(d),rank=False, additive=False, weigher=weigher))
print(weightedtau(np.argsort(x),np.argsort(y),rank=False, additive=False, weigher=weigher))

WeightedTauResult(correlation=0.6962121412338664, pvalue=nan)
WeightedTauResult(correlation=0.07118840640971633, pvalue=nan)
WeightedTauResult(correlation=0.18079986971236403, pvalue=nan)
